# Collections

## Сначала пробежимся по типам коллекций
![](table.drawio.svg)

## Вспомним сложности операции со стандартными коллекциями
Стоимость:
* расходы памяти на i-ый элемент - ???
* доступ до i-го элемента - ???
* поиск по значению - ???
* вставка - ???
* удаление - ???
* сортировка - ???
* расход памяти на массив - ???

<details>
<summary>Array</summary>

* расходы памяти на i-ый элемент - sizeof(T)
* доступ до i-го элемента - O(1)
* поиск по значению - O(n), O(log(n)) если отсортирован
* вставка - нет такого
* удаление - нет такого
* сортировка - O(n*log(n))
* расход памяти на массив - O(n)

</details>

<details>
<summary>List (Dynamic Array)</summary>

* расходы памяти на i-ый элемент - sizeof(T)
* доступ до i-го элемента - O(1)
* поиск по значению - O(n), O(log(n)) если отсортирован
* вставка - O(1) при добавлении в конец, если не превышает capacity. Иначе O(n)
* удаление - O(1), если в конце, иначе O(n)
* сортировка - O(n*log(n))
* расход памяти на массив - O(n)

</details>

<details>
<summary>LinkedList (Double Linked List)</summary>

* расходы памяти на i-ый элемент - sizeof(T) + sizeof(IntPtr)*2 (плюс ссылка на лист)
* доступ до i-го элемента - O(n)
* поиск по значению - O(n)
* вставка - O(1) в конец и начало, иначе O(n)
* удаление - O(1), если в конце и начале, иначе O(n)
* сортировка - O(n*log(n))
* расход памяти на массив - O(n)

</details>

<details>
<summary>Queue</summary>

* расходы памяти на i-ый элемент - sizeof(T)
* доступ до i-го элемента - O(1) 
* поиск по значению - O(n)
* вставка - O(1) в начало.
* удаление - O(1) в конце.
* сортировка - нет такого
* расход памяти на массив - O(n)

</details>

<details>
<summary>Dictionary</summary>

* расходы памяти на i-ый элемент - sizeof(T)
* доступ до i-го элемента - O(1) 
* поиск по значению - O(n)
* вставка - O(1) в начало.
* удаление - O(1) в конце.
* сортировка - нет такого
* расход памяти на массив - O(n)

</details>

Тут можно прочитать про устройство словаря <br/>
https://blog.markvincze.com/back-to-basics-dictionary-part-2-net-implementation/



### Устройство Dictionary если останется время
Словари в шарпах очень круто сделаны, так как они редко обращаются к куче (Только при Resize).\
Все это за счет использования структур (Из-за чего они все таки прожорливые по памяти)

```c#
public class Dictionary<TKey,TValue>
{
    private struct Entry       
    {
        public int hashCode;
        public int next;
        public TKey key;
        public TValue value;
    }

    private int[] buckets;
    private Entry[] entries;
    private int count;

    private int freeList // Указатель на последнее свободное место;
    private int freeCount // Количество свободных дыр;



    private void Insert(TKey key, TValue value, bool add)
    {
        //Вычисляем сам хэш bucket
        int hashCode = comparer.GetHashCode(key) & 0x7FFFFFFF;

        // Зная хеш, находим позицию в bucket, разделяя ее на общий размер
        int targetBucket = hashCode % buckets.Length;

        // Проходимся по всем вхождениям в bucket, если i имеет -1, то значит,
        // что цепочка коллизий закончилась и нам остается только вставить значение.
        // Если мы все-таки нашли значение по ключу, то 
        for (int i = buckets[targetBucket]; i >= 0; i = entries[i].next) {
            if (entries[i].hashCode == hashCode && comparer.Equals(entries[i].key, key)) {
                // Если мы используем Add, то у нас вылетает исключение, так как в словарь
                // нельзя добавить объекты с одинаковым ключом.
                // Сеттер нам это позволяет сделать, то есть поменять или добавить ключ-значение.
                if (add) { 
                    ThrowHelper.ThrowArgumentException(ExceptionResource.Argument_AddingDuplicate);
                }
                // Обновляем наше значение
                entries[i].value = value;
                return;
            }
        }

        int index;
        if (freeCount > 0) {
            // У нас во время удаления могут возникать дыры в массиве entries. Чтобы попусту не
            // расходывать место в памяти, мы кэшируем последнее свободное место, чтобы после
            // туда можно было впихнуть наш entry
            index = freeList;
            freeList = entries[index].next;
            freeCount--;
        }
        // Дыр в массиве entries нет
        else {
            // Если все полностью забито, то выделяем массив по больше и рехешим.
            if (count == entries.Length)
            {
                Resize();
                targetBucket = hashCode % buckets.Length;
            }

            // Можем просто поместить entry в свободное место
            index = count;
            count++;
        }

        // Выставляем параметры новой entry 
        entries[index].hashCode = hashCode;
        entries[index].next = buckets[targetBucket]; // Вставляем его в самое начало цепочки коллизий
                                                     // Поэтому все buckets инициализированы -1.
        entries[index].key = key;
        entries[index].value = value;
        buckets[targetBucket] = index; // Обновляем bucket, чтобы он указывал на эту entry
    }
}
```

## System.Collections
![](objectColl.svg)

***Вопрос*** В чем проблема использовать необобщенные коллекции
<details>
<summary>Ответ</summary>

* Отсутствует типо безопасность
* Нужно руками проводить преобразование от object к типу
* Боксинг для value type

</details>

Долго мы не будем на этом останавливаться, так как они используются только в легаси

## System.Collections.Generic

### Array

```c#
// Просто плоский массив
int[] vector = { 1, 2, 3, 4 };
vector.Where(val => val % 2 == 0);        // Ok

// Многомерный массив (Multi Dimensional Array)
int[,] multiDimArray = { { 1, 2, 3, 4 }, { 1, 2, 3, 4 }, { 1, 2, 3, 4 }, { 1, 2, 3, 4 } };
multiDimArray.Where(val => val % 2 == 0);    // Compilation Error :: Does not contain definition where
multiDimArray[0].Where(val => val % 2 == 0); // Compilation Error :: Wrong number of idicies

// Массив массивов (Jagged Array)
int[][] jaggedArray = [[ 1, 2], [ 1, 2]];
jaggedArray.Where(val => val % 2 == 0);      // Compilation Error :: Does not contain definition where
jaggedArray[0].Where(val => val % 2 == 0);   // Ok
```

### Проведем бенчи

```c#
[SimpleJob(RuntimeMoniker.Net80)]
public class ArraysBenchmarking()
{
    int[] simpleArray;
    int[,] mdArray;
    int[][] jaggedArray;

    [GlobalSetup]
    public void Setup()
    {
        simpleArray = new int[1000000];
        mdArray = new int[1000, 1000];
        jaggedArray = new int[1000][];

        for(int i = 0; i < 1000; i++)
        {
            jaggedArray[i] = new int[1000];
            for (int j = 0; j  < 1000; j++)
            {
                var randomNumber = Random.Shared.Next(0, 2);
                jaggedArray[i][j] = randomNumber;
                mdArray[i, j] = randomNumber;
                simpleArray[i * 1000 + j] = randomNumber;
            }
        }
    }

    [Benchmark]
    public void SimpleArrayBenchmark()
    {
        long count = 0;
        for (int i = 0; i < 1000; i++)
        {
            for (int j = 0; j < 1000; j++)
            {
                count += simpleArray[i * 1000 + j];
            }
        }

        // Чтобы не заоптимайзил
        simpleArray[0] = (int) count;
    }

    [Benchmark]
    public void MDArrayBenchmark()
    {
        long count = 0;
        for (int i = 0; i < 1000; i++)
        {
            for (int j = 0; j < 1000; j++)
            {
                count += mdArray[i, j];
            }
        }

        mdArray[0, 0] = (int) count;
    }

    [Benchmark]
    public void JaggedArrayBenchmark()
    {
        long count = 0;
        for (int i = 0; i < 1000; i++)
        {
            for (int j = 0; j < 1000; j++)
            {
                count += jaggedArray[i][j];
            }
        }

        jaggedArray[0][0] = (int) count;
    }
}
```

```

BenchmarkDotNet v0.13.12, Windows 10 (10.0.19045.3930/22H2/2022Update)
AMD Ryzen 5 5600X, 1 CPU, 12 logical and 6 physical cores
.NET SDK 8.0.100
  [Host]   : .NET 8.0.0 (8.0.23.53103), X64 RyuJIT AVX2
  .NET 8.0 : .NET 8.0.0 (8.0.23.53103), X64 RyuJIT AVX2

Job=.NET 8.0  Runtime=.NET 8.0  

```
| Method               | Mean     | Error   | StdDev  |
|--------------------- |---------:|--------:|--------:|
| SimpleArrayBenchmark | 436.2 μs | 0.17 μs | 0.16 μs |
| MDArrayBenchmark     | 653.4 μs | 0.58 μs | 0.49 μs |
| JaggedArrayBenchmark | 444.4 μs | 0.70 μs | 0.62 μs |


### Выводы

* Лучше всего использовать плоский массив (T[]).
* Никогда не использовать многомерный массив (по моему мнению)
* Массив массивов можно иногда использовать (?)